#### Faiss 
Facebook AI Similarity Search (FAISS) is a library for efficient similarity searh and clustering of dense vector. It contains algorithm that search  in set of vector of any size, up to ines that posibbly do not fir in RAM. It also contain supporting code for evaluation and parameter tuning.

In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader= TextLoader("speech.txt")
document = loader.load()
text_splitter= CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs=text_splitter.split_documents(document)

In [24]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [29]:
embeddings=OllamaEmbeddings(model="deepseek-r1:1.5b")
db=FAISS.from_documents(docs, embeddings)

In [32]:
### querying

query= " how does the speaker describe the desired outcome of the war?"


In [33]:
# docs = db.similarity_search(query)
docs = db.similarity_search(query)
# doc[0]
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'

#### As a Retriver
We can also convert the vectorstore into a reteiver class. This allow us to easily use it in other Langchain methods, which largely work with retrivers

In [34]:
retiriver= db.as_retriever()
retiriver.invoke(query)
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'

#### Similarity Search With Score
There are some FAISS specifi method. One of them is similarity_search_with_score, which allow you to return not only the document but also the destance score of the query to them. The return distance score is L2 distance. Therefore, a lower score is better.

In [35]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='2af74180-6cca-44a1-a63b-d25d501c44a9', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
  np.float32(6615.0625)),
 (Document(id='8bd74332-d7d0-4216-932f-

In [36]:
embeddings_vector=embeddings.embed_query(query)
embeddings_vector

[-0.49495768547058105,
 1.483289122581482,
 -0.9528125524520874,
 -2.7882823944091797,
 0.6814067959785461,
 -3.5889732837677,
 0.08242391794919968,
 0.0026288956869393587,
 -0.9786031246185303,
 -0.4604223370552063,
 2.0240397453308105,
 -3.837883949279785,
 1.4625232219696045,
 -1.221648097038269,
 0.5111510753631592,
 -0.26344799995422363,
 -0.4308147728443146,
 0.30433857440948486,
 0.6051972508430481,
 -3.9963014125823975,
 1.023577332496643,
 -0.624019205570221,
 2.065023422241211,
 -1.1822986602783203,
 0.8977167010307312,
 1.6722612380981445,
 -0.4435645043849945,
 -1.4694812297821045,
 -4.592637538909912,
 2.9132373332977295,
 -3.602116346359253,
 -0.7654873728752136,
 0.8645865321159363,
 -1.4387776851654053,
 -2.326453924179077,
 -0.7639542818069458,
 -1.2400410175323486,
 1.0198085308074951,
 2.1477866172790527,
 -1.6897060871124268,
 0.10644779354333878,
 0.038055043667554855,
 -1.4837708473205566,
 -0.2527567446231842,
 0.6397215127944946,
 3.0513031482696533,
 -0.0310008

In [38]:
docs_score=db.similarity_search_by_vector(embeddings_vector)
docs_score


[Document(id='2af74180-6cca-44a1-a63b-d25d501c44a9', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(id='8bd74332-d7d0-4216-932f-0508c7b13520', metadata={'so

In [39]:
### Saving and loading
db.save_local("faiss_index")

In [42]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [43]:
docs

[Document(id='2af74180-6cca-44a1-a63b-d25d501c44a9', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(id='8bd74332-d7d0-4216-932f-0508c7b13520', metadata={'so